In [1]:
# import sympy
%display typeset

In [11]:
var(' S E I R gamma mu alpha beta')

(S, E, I, R, gamma, mu, alpha, beta)

In [12]:
dsdt = -beta*S*I
dedt = beta*S*I - mu*E
didt = mu*E - alpha*I - gamma*I
drdt = alpha*I

In [13]:
solve([dsdt,dedt,didt,drdt],[S,E,I,R])

[[S == r1, E == 0, I == 0, R == r2]]

In [5]:
jack=jacobian([dsdt,dedt,didt,drdt],[S,E,I,R])
jack

[       -I*beta              0        -S*beta              0]
[        I*beta            -mu         S*beta              0]
[             0             mu -alpha - gamma              0]
[             0              0          alpha              0]

In [6]:
cp = jack.characteristic_polynomial()
cp

x^4 + (I*beta + alpha + gamma + mu)*x^3 + (I*alpha*beta + I*beta*gamma + I*beta*mu - S*beta*mu + alpha*mu + gamma*mu)*x^2 + (I*alpha*beta*mu + I*beta*gamma*mu)*x

In [8]:
jack.eigenvalues();

compartimentos com infectados: E e I

In [7]:
F_cal = matrix([[beta*S*I],[0]])

In [8]:
V_cal_neg = matrix([[mu*E],[alpha*I + gamma*I]])

In [9]:
V_cal_pos = matrix([[beta*S*I],[mu*E]])

In [10]:
V_cal = V_cal_neg - V_cal_pos
V_cal

[        -I*S*beta + E*mu]
[I*alpha + I*gamma - E*mu]

In [11]:
F = jacobian(F_cal(S=1),[E,I])
F

[   0 beta]
[   0    0]

In [12]:
V = jacobian(V_cal(S=1),[E,I])
V

[           mu         -beta]
[          -mu alpha + gamma]

In [13]:
M = F*V.inverse()
M

[beta/(alpha - beta + gamma) beta/(alpha - beta + gamma)]
[                          0                           0]

In [14]:
M=M.simplify_full()
M

[beta/(alpha - beta + gamma) beta/(alpha - beta + gamma)]
[                          0                           0]

In [23]:
M.eigenvalues()


[beta/(alpha - beta + gamma), 0]

In [15]:
R0=M[0,0].simplify_full()
R0

beta/(alpha - beta + gamma)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
!pip install pandas

  Using cached https://files.pythonhosted.org/packages/fb/e4/828bb9c2474ff6016e5ce96a78220d485436d5468c23068f4f6c2eb9cff8/pandas-1.1.5.tar.gz
  Installing build dependencies ... done
  Running setup.py install for pandas ... done


In [9]:
import pandas as pd

In [ ]:
def modelo(xs,t,ps):
    try:
        gamma = ps['gamma'].value 
        mu = ps['mu'].value
        alpha = ps['alpha'].value
        beta = ps['beta'].value
    except:
        gamma, mu, alpha, beta = ps
        
    
    S, E, I, R = xs
    
    return[
        -beta*S*I, 
        beta*S*I - mu*E,
        mu*E - alpha*I - gamma*I,
        alpha*I     
    ]


def engine(t, x0, ps):
    S = 1 - np.sum(x0)
    x0 = np.insert(x0, 0, S)
    x = solve(modelo, x0, t, args = (ps,))
    return x